## Step1: Cleaning

In [11]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords 

#clean a review
def getCleanText(text):
    text=text.lower()
    text=text.replace("<br/><br/>"," ")
    
    #Init objects
    tokenizer=RegexpTokenizer(r'\w+')
    en_stopwords=set(stopwords.words('english'))
    ps=PorterStemmer()
    
    #Tokenize
    tokens=tokenizer.tokenize(text)
    new_tokens=[token for token in tokens if token not in en_stopwords]
    stemmed_tokens=[ps.stem(token) for token in new_tokens]
    cleaned_text=' '.join(stemmed_tokens)
    return cleaned_text

In [31]:
#get train dataset
import pandas as pd
import numpy as np
df=pd.read_csv("Train.csv")
print(type(df),df.shape)

<class 'pandas.core.frame.DataFrame'> (40000, 2)


In [32]:
train_data=df.values
print(type(train_data),train_data.shape)
X_train=train_data[:,0]
Y_train=train_data[:,1]
print(X_train.shape,Y_train.shape)

<class 'numpy.ndarray'> (40000, 2)
(40000,) (40000,)


In [14]:
X_clean=[getCleanText(i) for i in X_train]

In [16]:
#load test dataset
df=pd.read_csv("Test.csv")
print(type(df),df.shape)

<class 'pandas.core.frame.DataFrame'> (10000, 1)


In [18]:
X_test=df.values.reshape((-1,))
X_test_clean=[getCleanText(i) for i in X_test]

## Step2: Vectorization

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
x_vec=cv.fit_transform(X_clean)
print(x_vec.shape)

(40000, 65742)


In [24]:
xt_vec=cv.transform(X_test_clean)
print(xt_vec.shape)

(10000, 65742)


## Step3: Train Model

In [34]:
from sklearn.naive_bayes import MultinomialNB
mnb=MultinomialNB()
mnb.fit(x_vec,Y_train)
#predictions
yt_pred=mnb.predict(xt_vec)

In [35]:
print(yt_pred[:10])

['neg' 'neg' 'neg' 'neg' 'pos' 'neg' 'pos' 'neg' 'pos' 'pos']


In [36]:
y_pred=mnb.predict(x_vec)
print(np.sum(Y_train==y_pred)/Y_train.shape[0])

0.8904


In [41]:
ans=pd.DataFrame(yt_pred,columns=['Label'])
ans.index.name='Id'
ans.to_csv('submit.csv',index=True) 